In [ ]:
function mutation2(generation,AdjList, probMutation, n,k)
    newGeneration=[]
    for (i,chromosome) in enumerate(generation)
        newChromosome=chromosome[1]
        # change aleatoir. couleur de sommet 
        for u in chromosome
            p=rand()
            if p<probMutation
                available_colors = setdiff(1:k, [newChromosome[n] for n in AdjList[u]])
                if !isempty(available_colors)
                    # If there are available colors, choose one randomly
                    newChromosome[u] = rand(available_colors)
                else
                    # If no available colors, revert to a random valid color
                    newChromosome[u] = rand(1:k)
                end
            
            end
        end
        # verifie, que ca nous donne qqch bon
        for u=1:n
            needChange=false
            for v in AdjList[u]
                if mutatedChromosome[v]==mutatedChromosome[u]
                    needChange=true
                    break;
                end
            end
            if needChange
                mutatedChromosome[u]=availableColor(AdjList,mutatedChromosome,k,u)
            end
             
        end
        cnflct=countConflict(AdjList,newChromosome,n)
        push!(newGeneration,(newChromosome,cnflct))
    end

    return newGeneration;
end

In [ ]:
function trouver_indice(r, p)
    somme_cumulative = 0.0
    n = length(p)
    
    for i in 1:n
        somme_cumulative += p[i]
        if r <= somme_cumulative
            return i
        end
    end
    
    # Si r dépasse toutes les sommes cumulatives, retourne -1 ou une valeur indiquant que rien n'a été trouvé.
    return -1
end

function roueFortune(generation, n, probCrossover,enfantsSize)
    parents=[]
    F=sum(pair[2] for pair in generation)
    probaGeneration=Float64[(F-pair[2])/((n-1)*F) for pair in generation]

    for i in 1:enfantsSize
        r=rand()
        index=trouver_indice(r,probaGeneration)
        push!(parents,generation[index])
    end
    return parents
end

In [ ]:
using Pkg
using LinearAlgebra
using Random
using Plots
using Dates

function readGraph(file)

    #Lecture du fichier et creation structure de read_graph
    fileRead=open(file,"r")
    lines=readlines(fileRead)
    AdjList=Dict{Int,Vector{Int}}()
    n=0;
    for line in lines
        str=split(line," ")
        if(str[1]=="e")
            u=parse(Int,str[2])
            v=parse(Int,str[3])
            if haskey(AdjList,u)
                push!(AdjList[u],v)
            else
                AdjList[u]=[v]
            end

            if haskey(AdjList,v)
                push!(AdjList[v],u)
            else
                AdjList[v]=[u]
            end
        end
        if(str[1]=="p")
            n=parse(Int,str[3])
        end
    end
        return AdjList,n

end    

function randomColoring(n,k,seed=0)

    Random.seed!(seed)
    return rand(1:k,n)
end

function availableColor(adjecenceList,Colors,k,u)
    nbUsesColor=zeros(Int,k)
    for v in adjecenceList[u]
        if Colors[v]!=0
            nbUsesColor[Colors[v]]+=1
        end
    end
    for color in 1:k
        if(nbUsesColor[color]==0)
            return color
        end
    end
    return argmin(nbUsesColor)

    
end

function glouton(adjecenceList,sommetSortDegree,n,k)
    Colors=zeros(Int,n)
    for i in sommetSortDegree
        Colors[i]=availableColor(adjecenceList,Colors, k ,i)
    end
    return Colors
end

function countConflict(adjecenceList, Colors, n)
    count=0;
    for u in 1:n
        for v in adjecenceList[u]
            if(Colors[u]==Colors[v])
                count+=1
            end
        end
    end
    return count/2; 
end

function tournament(Parents,tournamentSize,popSize)
    """
        Faire tournement de taille tournamentSize
    """
    indexCandidats=rand(1:popSize,tournamentSize)
    candidats=Parents[indexCandidats]
    sort!(candidats, by=x->x[2])
    return candidats[1]
end

function crossover(Parents,AdjList, n)
    enfants=[]
    
    while true
        if length(Parents)==0
            break
        end

        if length(Parents)==1
            push!(enfants,Parents[1])
            break
        end
        
        indexA=rand(1:length(Parents))
        parentA=Parents[indexA][1]
        deleteat!(Parents,indexA)
        indexB=rand(1:length(Parents))
        parentB=Parents[indexB][1]
        deleteat!(Parents,indexB)

        seuil=rand(1:(length(parentA)-1))
        child1=parentA[1:seuil]
        append!(child1,parentB[seuil+1:end])
        child2=parentB[1:seuil];
        append!(child2,parentA[seuil+1:end])

        push!(enfants,(child1,countConflict(AdjList,child1,n)))
        push!(enfants,(child2,countConflict(AdjList,child2,n)))
    end 
    return enfants
    
end

function mutation(generation,AdjList,probMutation,n,k)
    newGeneration=[]
    for (i,chromosome) in enumerate(generation)
        mutatedChromosome=chromosome[1]
        for u=1:n
            if rand()<probMutation
                needChange=false
                for v in AdjList[u]
                    if(mutatedChromosome[v]==mutatedChromosome[u])
                        needChange=true
                        break;
                    end
                end
                if needChange
                    mutatedChromosome[u]=availableColor(AdjList,mutatedChromosome,k,u)
                end
            end  
        end
        cnflct=countConflict(AdjList,mutatedChromosome,n)
        push!(newGeneration,(mutatedChromosome,cnflct))
    end
    return newGeneration
end



function Genetic(popSize, nbGenerationMax, probMutation, tauxElitisme,tournamentSize, AdjList,degreeSort,n,k)
    startTime=now()
    Random.seed!(42)

    #plot data
    fitnessBest=[]
    fitnessAvg=[]

    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,degreeSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end

    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<20&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection 
        
        selectedParents=[]
        #Tournament Selection
        
        # for i in 1:floor(tauxElitisme*popSize)
        #     push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        # end

        #Roue de la Fortune Selection
        
        selectedParents=roueFortune(Generation,popSize,tauxElitisme,ceil(tauxElitisme*popSize))

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end

        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    return bestSolution,fitnessBest,fitnessAvg,nbGeneration;

end


In [1]:

file="le450_15c.txt.txt"
k=15
tournamentSize=2
tauxElitisme=0.1
probMutation=0.2
nbGenerationMax=100000
popSize=100

AdjList,n=readGraph(file)

#Degree de chaque sommet
degree=zeros(Int,n)
for i in 1:n
    degree[i]=size(AdjList[i])[1]
end

sommetsSort=sortperm(degree,rev=true)
gloutonSolution=glouton(AdjList,sommetsSort,n,k)

println("Glouton solution: ",countConflict(AdjList,gloutonSolution,n))

geneticColoring, fitBest,fitAvg, nbGen=Genetic(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,sommetsSort,n,k)
println(countConflict(AdjList,geneticColoring,n))
#plot([1:nbGeneration],fitnessAvg,xlabel="nb Generations",ylabel="Avg fitness in generation",title="nbGeneration vs AVGFitness")

In [ ]:
 plt=plot([1:nbGen],fitAvg,title="nbGeneration vs BestFitness")
    xlabel!("nb Generations")
    ylabel!("Best fitness in generation")
    display(plt)

In [ ]:

function GeneticTabou(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,degree,n,k)
    startTime=now()
    Random.seed!(42)

    #plot data
    fitnessBest=[]
    fitnessAvg=[]

    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,degreeSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end

    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<20&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection 
        
        selectedParents=[]
        #Tournament Selection
        
        # for i in 1:floor(tauxElitisme*popSize)
        #     push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        # end

        #Roue de la Fortune Selection
        
        selectedParents=roueFortune(Generation,popSize,tauxElitisme,ceil(tauxElitisme*popSize))

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end

        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    return bestSolution,fitnessBest,fitnessAvg,nbGeneration;

end
